In [2]:
from itertools import chain
import pandas as pd
import os

citation_per_record = False
HCI_ori_paper_pd = pd.read_csv('paperinformation_HCI.tsv')
HCI_patent_pd = pd.read_csv('patent_information_HCI.tsv')
HCI_papercitationscience_pd = pd.read_csv('papercitationscience_CHI.tsv')
for conf in ["CSCW", "UbiComp", "UIST"]:
    new_df = pd.read_csv('papercitationscience_{}.tsv'.format(conf))
    HCI_papercitationscience_pd = HCI_papercitationscience_pd.append(new_df, ignore_index=True)
HCI_papercitationscience_pd = HCI_papercitationscience_pd.drop_duplicates()

# HCI_papercitationscience_pd = HCI_papercitationscience_pd[['paperid','patent']]
HCI_patent_pd = HCI_patent_pd[['patent_id', 'country', 'date', 'abstract', 'title', 'kind', 'num_claims', 'withdrawn', 'inventor_id', 'inventor_name_first', 'inventor_name_last', 'male_flag', 'attribution_status', 'assignee_id', 'assignee_name_first', 'assignee_name_last', 'organization']]

def get_patentid(x):
    x = x[x.index('-')+1:]
    if '-' in x: return x[:x.index('-')]
    else: return x
    
HCI_patent_pd["patent_id"] = HCI_patent_pd["patent_id"].astype(str)
HCI_papercitationscience_pd["patent"] = HCI_papercitationscience_pd["patent"].astype(str)
HCI_papercitationscience_pd["patent"] = HCI_papercitationscience_pd["patent"].apply(get_patentid)

/tmp/ipykernel_1063539/3392362829.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  HCI_papercitationscience_pd = HCI_papercitationscience_pd.append(new_df, ignore_index=True)
/tmp/ipykernel_1063539/3392362829.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  HCI_papercitationscience_pd = HCI_papercitationscience_pd.append(new_df, ignore_index=True)
/tmp/ipykernel_1063539/3392362829.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  HCI_papercitationscience_pd = HCI_papercitationscience_pd.append(new_df, ignore_index=True)


In [ ]:
HCI_ori_paper_pd.head(5)
from icecream import ic
HCI_ori_paper_pd["patents"] = ""
# HCI_ori_paper_pd = HCI_ori_pd.merge(patent_assignee_pd, left_on='paperid', right_on='paperid')
# from icecream import ic
for index, row in HCI_ori_paper_pd.iterrows():
    citing_patent_id = HCI_papercitationscience_pd.loc[HCI_papercitationscience_pd['magid']==row['mag_id'], 'patent']
    if not citing_patent_id.empty:
        patent_information_dict = {}
        for patent_id in citing_patent_id.values:
            # ic(patent_id)
            # ic(HCI_patent_pd.loc[HCI_patent_pd['patent_id']==9589072])
            patent_information_dict[patent_id] = {}
            patent_information_dict[patent_id]["patent_info"] = HCI_patent_pd.loc[HCI_patent_pd['patent_id']==str(patent_id),['country', 'date', 'abstract', 'title', 'kind', 'num_claims', 'withdrawn', 'assignee_id', 'assignee_name_first', 'assignee_name_last', 'organization']].to_dict('record')
            inventor_part = HCI_patent_pd.loc[HCI_patent_pd['patent_id']==str(patent_id), ['inventor_id', 'inventor_name_first', 'inventor_name_last', 'male_flag', 'attribution_status']].to_dict('record')
            patent_information_dict[patent_id]["inventors"] = inventor_part
        # ic(patent_information_dict)
        HCI_ori_paper_pd.at[index,'patents'] = patent_information_dict
HCI_ori_paper_pd = HCI_ori_paper_pd.drop_duplicates()
HCI_ori_paper_pd.to_csv('data4analysis/paper_patent_all_info_{}_per_record.tsv'.format("citation" if citation_per_record else "paper"), index=False)    

/tmp/ipykernel_2958873/9256764.py:14: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  patent_information_dict[patent_id]["patent_info"] = HCI_patent_pd.loc[HCI_patent_pd['patent_id']==str(patent_id),['country', 'date', 'abstract', 'title', 'kind', 'num_claims', 'withdrawn', 'assignee_id', 'assignee_name_first', 'assignee_name_last', 'organization']].to_dict('record')
/tmp/ipykernel_2958873/9256764.py:15: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  inventor_part = HCI_patent_pd.loc[HCI_patent_pd['patent_id']==str(patent_id), ['inventor_id', 'inventor_name_first', 'inventor_name_last', 'male_flag', 'attribution_status']].to_dict('record')
/tmp/ipykernel_2958

In [2]:
from itertools import chain
import pandas as pd
import os

# paper per record: paper_patent_all_info.tsv
# citation per record
HCI_all_pd = pd.read_csv('data4analysis/paper_patent_all_info_{}_per_record.tsv'.format("citation" if citation_per_record else "paper"))
HCI_affiliation_pd = pd.read_csv('HCI_paper_authorid_affiliation.tsv', sep='\t')
HCI_affiliation_pd = HCI_affiliation_pd.rename(columns={"paperid":"aff_paperid", "Affiliation": "aff_affiliations", "authorid": "aff_authorids"})


In [4]:
HCI_all_pd = HCI_all_pd.merge(HCI_affiliation_pd, left_on='mag_id', right_on='aff_paperid')
HCI_all_pd = HCI_all_pd.drop_duplicates()
HCI_all_pd.to_csv("data4analysis/final_all_HCI_info_{}_per_record.tsv".format("citation" if citation_per_record else "paper"), index=False)

In [5]:
HCI_all_pd.head(5)

,mag_id,paperid,externalIds,url,title,abstract,venue,year,referenceCount,citationCount,...,publicationDate,journal,authors,patents,aff_affiliations_x,aff_authorids_x,aff_paperid_x,aff_affiliations_y,aff_authorids_y,aff_paperid_y
0,2243662716,c6ad20de38825d9c03540bcddbaef13b17b03f9f,"{'DBLP': 'conf/ihc/PereiraGS14', 'MAG': '22436...",https://www.semanticscholar.org/paper/c6ad20de...,Cultura importa e faz diferença: uma discussão...,The Grand Challenges prospected in GranDIHC-BR...,IHC,2014.0,8.0,5.0,...,2014-10-27,{'pages': '469-472'},"{'offset': 0, 'data': [{'authorId': '144789947...",NaN,['Universidade do Estado de Santa Catarina (UD...,"['2112304474', '2167914822', '2207961889']",2243662716,['Universidade do Estado de Santa Catarina (UD...,"['2112304474', '2167914822', '2207961889']",2243662716
1,2244265604,b86ea014042285465850ca0002f9c2cfd838e694,"{'DBLP': 'conf/ihc/BarrosPM14', 'MAG': '224426...",https://www.semanticscholar.org/paper/b86ea014...,Uma investigação sobre as técnicas de visualiz...,In the last years a big growth in Information ...,IHC,2014.0,6.0,0.0,...,2014-10-27,{'pages': '397-400'},"{'offset': 0, 'data': [{'authorId': '31993475'...",NaN,"['Universidade Federal de Minas Gerais (UFMG),...","['2111590950', '2187427419', '2721250540']",2244265604,"['Universidade Federal de Minas Gerais (UFMG),...","['2111590950', '2187427419', '2721250540']",2244265604
2,2244428935,f634a48ef7d4d92fb03b8cc97701a6820b86fd36,"{'ArXiv': '1510.08554', 'DBLP': 'conf/chi/Ruot...",https://www.semanticscholar.org/paper/f634a48e...,"""We're on the Same Page"": A Usability Study of...",Secure email is increasingly being touted as u...,CHI,2015.0,26.0,77.0,...,2015-10-29,{'name': 'Proceedings of the 2016 CHI Conferen...,"{'offset': 0, 'data': [{'authorId': '2554572',...",NaN,['Brigham Young University & Sandia National L...,"['1140008411', '1833198927', '2176732160', '22...",2244428935,['Brigham Young University & Sandia National L...,"['1140008411', '1833198927', '2176732160', '22...",2244428935
3,2244483578,d9285ea0a51648defeff6af40f8fade7c30a342f,"{'MAG': '2244483578', 'DBLP': 'conf/ihc/PretiF...",https://www.semanticscholar.org/paper/d9285ea0...,Interação em nuvens,The growth of communication and information te...,IHC,2010.0,8.0,3.0,...,2010-10-05,{'pages': '209-212'},"{'offset': 0, 'data': [{'authorId': '209737199...",NaN,"['UFMT, Cuiabá - MT#TAB#', 'Escola Politécnica...","['2053971422', '568124060']",2244483578,"['UFMT, Cuiabá - MT#TAB#', 'Escola Politécnica...","['2053971422', '568124060']",2244483578
4,2244508988,d305e61ca0be4c308e1b60272ec7fd279c4b6d71,"{'DBLP': 'conf/ihc/RamosB14', 'MAG': '22445089...",https://www.semanticscholar.org/paper/d305e61c...,Desafios da computação ubíqua por uma visão de...,Ubiquitous computing is characterized by the t...,IHC,2014.0,12.0,1.0,...,2014-10-27,{'pages': '429-432'},"{'offset': 0, 'data': [{'authorId': '121029437...",NaN,"['UTFPR, Curitiba PR Brasil#TAB#', 'UTFPR, Cur...","['2686308945', '2714009501']",2244508988,"['UTFPR, Curitiba PR Brasil#TAB#', 'UTFPR, Cur...","['2686308945', '2714009501']",2244508988


In [39]:
from itertools import chain
import pandas as pd
import os


In [40]:
HCI_all_pd = pd.read_csv('final_all_HCI_info_{}_per_record.tsv'.format("citation" if citation_per_record else "paper"))

In [48]:
# text_file = open("newHCIdata/debug.txt", "w")
# text_file.write(HCI_all_pd["patents"][26303])
import json
json.loads(json.loads(json.loads(HCI_all_pd["patents"][26303])["7134756"])["inventors"])[0]["inventor_id"]

'fl:st_ln:drucker-3'